
<strong>Objective:</strong>
Today, I am to refactor the Timing and try to create a base class  that can be used for other top-level blocks. 


Below is a general structure, that I like for the Blocks
class. However, the question is if we could extend this 
to other classes by refactoring this into a base class 
and let other classes inherit it? 

<strong>NOTE</strong>: In order to judge if a class is correctly constructed, we must
have an *objective* of the class. Without its objectives how could we 
judge it? 

In [14]:
class PatternBurstBlocks(object): 
    def __init__(self): 
        self.patternBursts = {} # Name -> PatternBurstObject
    def add(self, patternBurst): 
        if not isinstance(patternBurst, PatternBurst): 
            raise ValueError("Must provide instance of PatternBurst.")
        self.patternBursts[patternBurst.name] = patternBurst
    def get(self, name): 
        if name in self.patternBursts: return self.patternBursts[name]
        else: return None
    def __len__(self): 
        return len(self.patternBursts)

pbbs = PatternBurstBlocks()


The `self.patternBursts = {}` could be renamed so its reuse doesnt 
lead to loss of information. We could rename to `self.objects={}`.
- The internal dictionary is a mapping of names (domain names) to 
thier respective objects. 
- This implies that no domain names be reused or that all domain 
names be unique (which seems to be what I have seen within the 
standard).

Of the other methods, `add` seems to the only one that would need 
to be implementated at the inhertied level. This is because we 
want to assess the class type of the added object. This class type 
will change depending on the class. 

So, by discovering the common theme across all top-level blocks, we can create our base class, `Blocks`.

In [ ]:
class Blocks(object): 
    def __init__(self): 
        self.objects = {} # Name -> Object
        
    def add(self, entity, classtype): 
        if not isinstance(entity, classtype): 
            raise ValueError("Must provide instance of %s."%(classtype))
        self.objects[entity.name] = entity
        
    def get(self, name): 
        if name in self.objects: return self.objects[name]
        else: return None
        
    def __len__(self): 
        return len(self.objects)   

In [19]:
class PatternBurst(object):
    def __init__(self, name): 
        self.name = name 
        self.blocks = {"PatList": [], "ParallelPatList":[], "PatSet":[]}
        self.ordering = [] # [(type,index)]

class PatternBurstBlocks(Blocks): 
    def __init__(self): 
        super().__init__()
    def add(self, patternBurst): 
        super().add(patternBurst, PatternBurst)

pbbs  = PatternBurstBlocks()
pb = PatternBurst(name="XYZ")
pbbs.add(pb)
#pbbs.add({"hey":8})
print(pbbs.get("XYZ"))


Now that we have shown how the `Blocks` base class can be used for a real top-level class, we can now easily create the containers for every other class. 

The key is to remember that each instance needs to override the add method and have the class type created. The only exception would possibly be the `Signals` block. That is because, according to 1450.0, only one Signals block will be referenced during translation; the first one. All others will be ignored.  

In [22]:
class TimingBlocks(Blocks): 
    def __init__(self): 
        super().__init__()
    def add(self, timing): 
        super().add(timing, Timing)
        
class DCLevelsBlocks(Blocks): 
    def __init__(self): 
        super().__init__()
    def add(self, dclevels): 
        super().add(dclevels, DCLevels)

class SignalGroupsBlocks(Blocks): 
    def __init__(self): 
        super().__init__()
    def add(self, signalgroups): 
        super().add(signalgroups, SignalGroups)
        
        
        
        
        

# Conclusion: 
It seems that we can you use the `Blocks` class as a base for all other toplevel container classes. This allows for maximum code resuse. But more importanly, it does not interfere with the customization that each class may requires. This base class is only meant to service the purpose of standrdizing the 'container' like aspects of top-level blocks found in STIL. 